In [74]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd
from urllib.parse import urlencode, parse_qs, urlparse
import json
import os

region_code = '2'

# Предопределённые диапазоны цен (расширены до 30 млн ₽ для полноты)
price_ranges = [
    (0, 500000), (500000, 1000000), (1000000, 1500000), (1500000, 2000000),
    (2000000, 2500000), (2500000, 3000000), (3000000, 4000000), (4000000, 5000000),
    (5000000, 6000000), (6000000, 7000000), (7000000, 8000000), (8000000, 9000000),
    (9000000, 10000000), (10000000, 12000000), (12000000, 14000000), (14000000, 16000000),
    (16000000, 18000000), (18000000, 20000000), (20000000, 25000000), (25000000, 30000000)
]
max_price_limit = 300000000  # Установлен предел в 300 млн ₽

# Настройка браузера
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(options=options)

# Функции сохранения и загрузки
def save_state(room, current_max_price):
    state = {"room": room, "current_max_price": current_max_price}
    with open("parse_state.json", "w") as f:
        json.dump(state, f)

def load_state():
    if os.path.exists("parse_state.json"):
        with open("parse_state.json", "r") as f:
            return json.load(f)
    return None

def save_data(data):
    df = pd.DataFrame(data).drop_duplicates(subset="id")
    df.to_csv("offers_data.csv", index=False)

# Авторизация
base_url = "https://spb.cian.ru/cat.php"
params = {"deal_type": "sale", "offer_type": "flat", "region": region_code, "engine_version": "2"}
initial_url = f"{base_url}?{urlencode(params)}"
driver.get(initial_url)
print("Пожалуйста, авторизуйтесь на сайте Циан в открывшемся браузере.")
print("После авторизации введите 'y' в консоль и нажмите Enter для продолжения.")

while True:
    user_input = input("Вы авторизовались? (y/n): ")
    if user_input.lower() == 'y':
        print("Авторизация подтверждена. Продолжаем парсинг...")
        break
    elif user_input.lower() == 'n':
        print("Пожалуйста, авторизуйтесь и введите 'y' после этого.")
    else:
        print("Введите 'y' для продолжения или 'n', если ещё не авторизовались.")

WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "[data-name='SummaryHeader'] h5"))
)
total_offers_text = driver.find_element(By.CSS_SELECTOR, "[data-name='SummaryHeader'] h5").text
total_offers = int(re.search(r'\d[\d\s]*', total_offers_text).group().replace(" ", ""))
print(f"Общее количество объявлений по Городу: {total_offers}")

# Базовые параметры
base_params = {"currency": "2", "deal_type": "sale", "engine_version": "2", "m2": "1", "offer_type": "flat", "region": region_code}
room_types = ["room1", "room2", "room3", "room4", "room5", "room6", "room7", "room9"]

# Загрузка данных
all_data = []
if os.path.exists("offers_data.csv"):
    df_existing = pd.read_csv("offers_data.csv")
    all_data = df_existing.to_dict("records")
    print(f"Загружено {len(all_data)} существующих записей из offers_data.csv")

state = load_state()
start_room = state["room"] if state else room_types[0]
start_max_price = state["current_max_price"] if state else 0
start_index = room_types.index(start_room) if start_room in room_types else 0

for room in room_types[start_index:]:
    print(f"\nОбрабатываем тип комнат: {room}")
    room_params = base_params.copy()
    room_params[room] = "1"
    room_url = f"{base_url}?{urlencode(room_params)}"
    driver.get(room_url)
    time.sleep(5)

    WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-name='SummaryHeader'] h5"))
    )
    room_offers_text = driver.find_element(By.CSS_SELECTOR, "[data-name='SummaryHeader'] h5").text
    room_total_offers = int(re.search(r'\d[\d\s]*', room_offers_text).group().replace(" ", ""))
    print(f"Общее количество для {room}: {room_total_offers}")
    room_data = []  # Инициализируем пустой список для каждого типа комнат
    empty_banner_count = 0  # Счётчик подряд идущих баннеров

    # Обработка каждого диапазона
    for base_min_price, base_max_price in price_ranges:
        if base_min_price < start_max_price and room == start_room:
            continue  # Пропускаем уже обработанные диапазоны
        min_price = base_min_price
        while min_price < base_max_price:
            max_price = base_max_price  # Начинаем с полного остатка диапазона
            range_data = []  # Временный список для текущего поддиапазона
            while True:
                params = room_params.copy()
                params["minprice"] = str(min_price)
                params["maxprice"] = str(max_price)
                check_url = f"{base_url}?{urlencode(params)}"
                print(f"Проверяем URL: {check_url}")
                driver.get(check_url)
                time.sleep(5)

                try:
                    # Ждём либо заголовок с количеством, либо баннер
                    WebDriverWait(driver, 30).until(
                        lambda driver: driver.find_elements(By.CSS_SELECTOR, "[data-name='SummaryHeader'] h5") or 
                                       driver.find_elements(By.CSS_SELECTOR, "[data-name='EmptyListingBanner']")
                    )
                    # Проверяем, есть ли баннер
                    if driver.find_elements(By.CSS_SELECTOR, "[data-name='EmptyListingBanner']"):
                        print(f"Диапазон {min_price}–{max_price}: нет объявлений (баннер)")
                        offers_count = 0
                        empty_banner_count += 1  # Увеличиваем счётчик баннеров
                    else:
                        offers_text = driver.find_element(By.CSS_SELECTOR, "[data-name='SummaryHeader'] h5").text
                        offers_count = int(re.search(r'\d[\d\s]*', offers_text).group().replace(" ", ""))
                        print(f"Диапазон {min_price}–{max_price}: {offers_count} объявлений")
                        empty_banner_count = 0  # Сбрасываем счётчик, если есть объявления
                except Exception as e:
                    print(f"Ошибка при проверке диапазона {min_price}–{max_price}: {e}")
                    offers_count = 0
                    empty_banner_count += 1  # Считаем это баннером, если ошибка

                if offers_count > 1512:
                    new_max_price = min_price + (max_price - min_price) // 2
                    print(f"Слишком много объявлений ({offers_count} > 1512), делим диапазон: {min_price}–{new_max_price}")
                    max_price = new_max_price
                    continue
                else:
                    break

            # Проверяем, не пора ли перейти к следующей комнате
            if empty_banner_count >= 2:
                print(f"Баннер появился 2 раза подряд. Переходим к следующему типу комнат ({room} завершён).")
                break  # Прерываем цикл по min_price
            # Если два баннера подряд, также прерываем внешний цикл по base_min_price
            if empty_banner_count >= 2:
                break

            # Парсинг страниц, только если есть объявления
            if offers_count > 0:
                page = 1
                while page <= 54:
                    params["p"] = str(page)
                    page_url = f"{base_url}?{urlencode(params)}"
                    for attempt in range(3):
                        try:
                            driver.get(page_url)
                            time.sleep(5)
                            current_url = driver.current_url
                            requested_page = str(page)
                            parsed_url = urlparse(current_url)
                            current_params = parse_qs(parsed_url.query)
                            current_page = current_params.get("p", ["1"])[0]

                            if current_page != requested_page and current_page == "1" and page > 1:
                                print(f"Редирект на первую страницу (p=1) при запросе p={page}. Данные закончились.")
                                page = 55  # Выходим из цикла страниц
                                break

                            WebDriverWait(driver, 30).until(
                                EC.presence_of_element_located((By.CLASS_NAME, "_93444fe79c--container--Povoi"))
                            )
                            listings = driver.find_elements(By.CLASS_NAME, "_93444fe79c--container--Povoi")
                            for listing in listings:
                                title = listing.find_element(By.CSS_SELECTOR, "[data-mark='OfferTitle']").text
                                link = listing.find_element(By.CSS_SELECTOR, "a._93444fe79c--media--9P6wN").get_attribute("href")
                                offer_id = link.split("/")[-2]
                                subtitle = listing.find_element(By.CSS_SELECTOR, "[data-mark='OfferSubtitle']").text if listing.find_elements(By.CSS_SELECTOR, "[data-mark='OfferSubtitle']") else None
                                price_elem = listing.find_element(By.CSS_SELECTOR, "[data-mark='MainPrice']").text
                                price = int(re.sub(r'[^\d]', '', price_elem)) if price_elem else None
                                price_per_m2_elem = listing.find_element(By.CSS_SELECTOR, "[data-mark='PriceInfo']").text
                                price_per_m2 = int(re.sub(r'[^\d]', '', price_per_m2_elem)) if price_per_m2_elem else None
                                geo_labels = listing.find_elements(By.CSS_SELECTOR, "[data-name='GeoLabel']")
                                address = ", ".join([label.text for label in geo_labels]) if geo_labels else None
                                metro_elem = listing.find_elements(By.CSS_SELECTOR, "._93444fe79c--container--w7txv a")
                                metro = metro_elem[0].find_element(By.XPATH, "./div[2]").text if metro_elem else None
                                metro_distance = listing.find_element(By.CSS_SELECTOR, "._93444fe79c--remoteness--q8IXp").text if listing.find_elements(By.CSS_SELECTOR, "._93444fe79c--remoteness--q8IXp") else None
                                floor_pattern = r'(\d+)/(\d+)\s*этаж'
                                floor_match = re.search(floor_pattern, title) or (subtitle and re.search(floor_pattern, subtitle))
                                floor = int(floor_match.group(1)) if floor_match else None
                                total_floors = int(floor_match.group(2)) if floor_match else None
                                area_pattern = r'(\d+[,\.]\d+|\d+)\s*м²'
                                area_match = re.search(area_pattern, subtitle or title)
                                area = float(area_match.group(1).replace(',', '.')) if area_match else None
                                rooms_pattern = r'(\d+)-комн\.\s*(квартира|апартаменты)?'
                                rooms_match = re.search(rooms_pattern, subtitle or title)
                                rooms_count = int(rooms_match.group(1)) if rooms_match else None
                                property_type = rooms_match.group(2) if rooms_match and rooms_match.group(2) else None
                                seller_block = listing.find_element(By.CLASS_NAME, "_93444fe79c--agent--HG9xn")
                                seller_type = seller_block.find_element(By.CSS_SELECTOR, "span._93444fe79c--color_gray60_100--r_axa").text
                                seller_name_elem = seller_block.find_elements(By.CSS_SELECTOR, "a._93444fe79c--link--wbne1")
                                seller_name = seller_name_elem[0].text if seller_name_elem else None
                                documents_verified = bool(seller_block.find_elements(By.XPATH, ".//span[contains(text(), 'Документы проверены')]"))
                                super_agent = bool(seller_block.find_elements(By.XPATH, ".//span[contains(text(), 'Суперагент')]"))
                                rosreestr_verified = bool(listing.find_elements(By.XPATH, ".//span[contains(text(), 'Проверено в Росреестре')]"))
                                zhk_elem = listing.find_elements(By.CSS_SELECTOR, "a._93444fe79c--jk--dIktL")
                                zhk_name = zhk_elem[0].text if zhk_elem else None
                                desc_elem = listing.find_elements(By.CLASS_NAME, "_93444fe79c--description--SqTNp")
                                description = desc_elem[0].text if desc_elem else None

                                range_data.append({
                                    "id": offer_id, "title": title, "link": link, "room": room, "subtitle": subtitle,
                                    "price": price, "price_per_m2": price_per_m2, "address": address,
                                    "metro": metro, "metro_distance": metro_distance, "floor": floor, "total_floors": total_floors,
                                    "area": area, "rooms_count": rooms_count, "property_type": property_type,
                                    "seller_type": seller_type, "seller_name": seller_name,
                                    "documents_verified": documents_verified, "super_agent": super_agent,
                                    "rosreestr_verified": rosreestr_verified, "zhk_name": zhk_name, "description": description
                                })
                            print(f'Парсинг страницы {page} выполнен успешно')
                            break
                        except Exception as e:
                            print(f"Ошибка загрузки страницы {page} (попытка {attempt+1}/3): {e}")
                            if attempt < 2:
                                time.sleep(15)
                            else:
                                print(f"Не удалось загрузить страницу {page} после 3 попыток. Пропускаем.")
                                break
                    page += 1

                # Добавляем данные текущего диапазона в room_data и all_data
                print(f"Собрано {len(range_data)} записей для диапазона {min_price}–{max_price}")
                room_data.extend(range_data)
                all_data.extend(range_data)
                print(f"Текущая длина room_data: {len(room_data)}, all_data: {len(all_data)}")
                save_data(all_data)
                save_state(room, max_price)
                print(f"Промежуточное сохранение: данные в offers_data.csv, состояние в parse_state.json")
            else:
                print(f"Пропускаем парсинг страниц для диапазона {min_price}–{max_price} — объявлений нет")
                save_state(room, max_price)

            min_price = max_price
            time.sleep(5)

            if max_price >= max_price_limit:
                break

        if empty_banner_count >= 2:
            break  # Прерываем цикл по base_min_price и переходим к следующей комнате

    room_all_data = [d for d in all_data if d["room"] == room]
    unique_room_count = len(set([d["id"] for d in room_all_data]))
    print(f"Собрано для {room} (всего): {unique_room_count} из {room_total_offers}")
    print(f"Потери для {room}: {room_total_offers - unique_room_count}")

# Итоги
df = pd.DataFrame(all_data).drop_duplicates(subset="id")
unique_total_count = len(df)
print(f"\nИтоговое количество уникальных объявлений: {unique_total_count}")
print(f"Общее количество объявлений по городу: {total_offers}")
print(f"Общие потери: {total_offers - unique_total_count}")

driver.quit()
print(df)

Пожалуйста, авторизуйтесь на сайте Циан в открывшемся браузере.
После авторизации введите 'y' в консоль и нажмите Enter для продолжения.
Авторизация подтверждена. Продолжаем парсинг...
Общее количество объявлений по Городу: 60122
Загружено 7889 существующих записей из offers_data.csv

Обрабатываем тип комнат: room1
Общее количество для room1: 19790
Проверяем URL: https://spb.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&m2=1&offer_type=flat&region=2&room1=1&minprice=2000000&maxprice=2500000
Диапазон 2000000–2500000: нет объявлений (баннер)
Пропускаем парсинг страниц для диапазона 2000000–2500000 — объявлений нет
Проверяем URL: https://spb.cian.ru/cat.php?currency=2&deal_type=sale&engine_version=2&m2=1&offer_type=flat&region=2&room1=1&minprice=2500000&maxprice=3000000
Диапазон 2500000–3000000: нет объявлений (баннер)
Баннер появился 2 раза подряд. Переходим к следующему типу комнат (room1 завершён).
Собрано для room1 (всего): 7889 из 19790
Потери для room1: 11901

Обрабатыв